# Exploring Listwise ranking models

Steps:
1. Generate random query data
2. Convert to listwise data representation
3. Train ranker models

Todo:
1. Try different query sizes
2. Introduce ranking/position bias

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-whitegrid")

np.random.seed(0)

/var/folders/_v/nlh4h1yx2n1gd6f3szjlgxt40000gr/T/ipykernel_55795/3873074974.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


Generate data from queries

using a set number of documents for each query.

In [145]:
n_queries = 10
n_documents = 5
n_documents_per_query = 3

rnd = np.random.default_rng()

# document IDs
query_data = np.array(
    [
        rnd.choice(np.arange(n_documents), size=(n_documents_per_query), replace=False)
        for _q in range(n_queries)
    ]
)
# labels are taken as the action on the document ID
labels_data = query_data[
    np.arange(n_queries), rnd.choice(np.arange(n_documents_per_query), size=(n_queries))
]

df = pd.DataFrame(
    data=np.concatenate(
        [np.arange(n_queries)[:, np.newaxis], query_data, labels_data[:, np.newaxis]],
        axis=1,
    ),
    columns=["query_id"]
    + [f"doc_{_idx}" for _idx in range(n_documents_per_query)]
    + ["label"],
)
df

,query_id,doc_0,doc_1,doc_2,label
0,0,4,0,1,1
1,1,0,4,1,4
2,2,0,1,4,0
3,3,4,0,2,4
4,4,0,1,4,4
5,5,3,1,2,1
6,6,1,2,3,3
7,7,1,0,3,3
8,8,1,0,2,2
9,9,4,3,1,1


Generate listwise data

In [147]:
for _idx,row in df.iterrows():
    row


# df_melt = df.melt(
#     id_vars="query_id",
#     value_vars=[f"doc_{_idx}" for _idx in range(n_documents_per_query)],
#     var_name="doc_id",
# ).sort_values(["query_id", "doc_id"])
row

query_id    9
doc_0       4
doc_1       3
doc_2       1
label       1
Name: 9, dtype: int64

Generate data from queries in listwise form

using a set number of documents for each query.

In [154]:
n_queries = 10
n_documents = 5
n_documents_per_query = 3

rnd = np.random.default_rng()

document_rows = []
for _q_id in range(n_queries):
    doc_ids = rnd.choice(
        np.arange(n_documents), size=(n_documents_per_query), replace=False
    )
    labels = rnd.uniform(size=(n_documents_per_query))
    labels = (labels == np.max(labels)) * 1
    for doc_id, label in zip(doc_ids, labels):
        document_rows.append({"query_id": _q_id, "doc_id": doc_id, "label": label})

df = pd.DataFrame(document_rows)
df

,query_id,doc_id,label
0,0,1,1
1,0,0,0
2,0,3,0
3,1,2,0
4,1,3,1
5,1,1,0
6,2,1,0
7,2,4,0
8,2,0,1
9,3,0,1


Build rankers

Assume that doc_id is the only feature (ordinal encoding)

In [155]:
import xgboost as xgb

In [176]:
ranker = xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:ndcg", lambdarank_pair_method="mean")
ranker.fit(X=df[['doc_id']], y=df['label'], qid=df['query_id'])

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='mean',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

Predictions

In [177]:
scores = ranker.predict(X=df[['doc_id']])
df = df.assign(scores=scores)
df

,query_id,doc_id,label,scores
0,0,1,1,0.646010
1,0,0,0,-0.761930
2,0,3,0,-0.155311
3,1,2,0,0.613090
4,1,3,1,-0.155311
5,1,1,0,0.646010
6,2,1,0,0.646010
7,2,4,0,-2.386494
8,2,0,1,-0.761930
9,3,0,1,-0.761930


Predictions by doc_id are always the same...?

In [178]:
df.groupby(["doc_id"])["scores"].agg(["max", "min", "count"])

,max,min,count
doc_id,,,
0,-0.761930,-0.761930,7
1,0.646010,0.646010,8
2,0.613090,0.613090,4
3,-0.155311,-0.155311,6
4,-2.386494,-2.386494,5


Counterfactual predictions

Estimate doc_id 1 relevance in queries of different lengths
Relevance score for query_id/doc_id pairs are the same...?

In [179]:
df_rand = df.sample(frac=0.5, replace=False).sort_values(["query_id", "doc_id"])
scores = ranker.predict(X=df_rand[["doc_id"]])
df_rand = df_rand.assign(scores=scores)

df_rand.merge(df, on=["query_id", "doc_id", "label"], how="outer").sort_values(
    ["query_id", "doc_id"]
).assign(diff=lambda x: x["scores_y"] - x["scores_x"])

,query_id,doc_id,label,scores_x,scores_y,diff
16,0,0,0,NaN,-0.761930,NaN
15,0,1,1,NaN,0.646010,NaN
17,0,3,0,NaN,-0.155311,NaN
20,1,1,0,NaN,0.646010,NaN
18,1,2,0,NaN,0.613090,NaN
19,1,3,1,NaN,-0.155311,NaN
21,2,0,1,NaN,-0.761930,NaN
0,2,1,0,0.646010,0.646010,0.0
1,2,4,0,-2.386494,-2.386494,0.0
2,3,0,1,-0.761930,-0.761930,0.0


Counterfactual predictions

Change the doc_ids for half of the queries

In [ ]:
df_rand = df.sample(frac=0.5, replace=False).sort_values(["query_id", "doc_id"])
scores = ranker.predict(X=df_rand[["doc_id"]])
df_rand = df_rand.assign(scores=scores)

df_rand.merge(df, on=["query_id", "doc_id", "label"], how="outer").sort_values(
    ["query_id", "doc_id"]
).assign(diff=lambda x: x["scores_y"] - x["scores_x"])

,query_id,doc_id,label,scores_x,scores_y,diff
0,0,0,0,-0.636212,-0.636212,0.0
1,0,1,1,0.465795,0.465795,0.0
2,0,3,0,-0.336154,-0.336154,0.0
3,1,1,0,0.465795,0.465795,0.0
15,1,2,0,NaN,0.472285,NaN
4,1,3,1,-0.336154,-0.336154,0.0
5,2,0,1,-0.636212,-0.636212,0.0
6,2,1,0,0.465795,0.465795,0.0
16,2,4,0,NaN,-1.019274,NaN
17,3,0,1,NaN,-0.636212,NaN


In [156]:
from sklearn.datasets import make_classification
import numpy as np

import xgboost as xgb

# Make a synthetic ranking dataset for demonstration
seed = 1994
X, y = make_classification(random_state=seed)
rng = np.random.default_rng(seed)
n_query_groups = 3
qid = rng.integers(0, 3, size=X.shape[0])

# Sort the inputs based on query index
sorted_idx = np.argsort(qid)
X = X[sorted_idx, :]
y = y[sorted_idx]
qid = qid[sorted_idx]
